In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')

# print(raw.ch_names)

# events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')
# picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')
# tmin, tmax = 1., 4.


# event_id = [ 768, 1791, 2323, 2324, 2325, 2326, 2337, 2339, 3346, 3347, 3348, 3349, 3360, 3362]
# epochs = mne.Epochs(raw, events, None, tmin, tmax, proj=True, picks=picks,
#                 baseline=None, preload=True)




In [ ]:
print(raw.ch_names[30])

In [ ]:
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

data = raw.get_data()
print(data.shape)


# raw = mne.io.read_raw_edf("k3b.gdf", preload=True)
_, pos, kind, chan, dur = raw.find_edf_events()
kind_hex = np.array([hex(n) for n in kind])

print(pos[1])
print(pos[2])
print(pos[:20])

print(len(kind_hex))
print(len(pos))
sampling_rate = 250


beep_pos = [pos[i] for i in range(len(kind_hex)) if kind_hex[i] in ['0x301','0x302','0x303','0x304']]
labels = [i[4] for i in kind_hex if i in ['0x301','0x302','0x303','0x304']]
print(labels)


def making_epochs(data):
    epochs = []
    for b in beep_pos:
        epochs.append(data[:,b+sampling_rate*1:b+sampling_rate*4])
    return np.asarray(epochs)


epochs = making_epochs(data)
# dif = [int(pos[i]) - int(pos[i-1]) for i in range(len(beep)-1) ]

# print(dif[:100])

In [ ]:
from sklearn.preprocessing import StandardScaler
import copy


epochs_std = copy.copy(epochs)


sample_num, chan_num, timepoint = epochs.shape
for c in range(chan_num):
    original_timepoints = epochs[:,c,:]
    scaler = StandardScaler()
    scaler.fit(original_timepoints)
    chan_std = scaler.transform(original_timepoints)
    epochs_std[:,c,:] = chan_std
    

print(np.array_equal(epochs_std, epochs))
print(epochs_std.shape)



In [ ]:
import sklearn 
import numpy as np
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(y_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
import pywt

testing_epochs = []
single_epoch = []
samples, chan_num, timesteps = epochs.shape
epochs_power = epochs**2   
# for s in range(samples):
#     single_epoch = []
#     single_epoch.append(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,30,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,33,:], 'db4', level=4)[0])
#     testing_epochs.append(np.asarray(single_epoch))
    
    
for s in range(samples):
    single_epoch = []
    single_epoch = np.asarray(single_epoch).reshape((1,-1))
    for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
        single_epoch = np.concatenate((single_epoch,np.asarray(xi).reshape((1,-1))), axis = 1)

    testing_epochs.append(single_epoch)
  

testing_epochs = np.asarray(testing_epochs)
print(testing_epochs.shape)
#9540

print(np.count_nonzero(abs(testing_epochs) > 1.0056865244277509e-9))
print(np.min(abs(testing_epochs)))


testing_epochs = testing_epochs.reshape((180, 775))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(testing_epochs)
# print(scaler.mean_)
testing_epochs = scaler.transform(testing_epochs)

print(testing_epochs.shape)

In [ ]:
# np.concatenate(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0], pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[1])

y = []
y = np.asarray(y).reshape((1,-1))
for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
    y = np.concatenate((y,np.asarray(xi).reshape((1,-1))), axis = 1)
    
# print(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0].shape)
#378 192 99  53  53
print(y.shape)


In [ ]:
weights, biases = model.layers[0].get_weights()
print(np.asarray(weights)[:,0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras import backend as K
import keras
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from keras import optimizers
from keras import regularizers

K.clear_session()

# testing_epochs = testing_epochs.reshape((180,1, 775))


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()


from keras.callbacks import EarlyStopping
stop_here_please = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)


testing_epochs = np.asarray(testing_epochs)

# testing_epochs = testing_epochs.swapaxes(1,2)

# time_steps = 53
# n_features = 3

time_steps = 775
n_features = 1



cv = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
scores = []
testing_epochs_ANN = testing_epochs.reshape(testing_epochs.shape[0], -1)
cvscores = []

for train_idx, test_idx in cv.split(testing_epochs):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = testing_epochs[train_idx]
    X_test = testing_epochs[test_idx]
    
    

    model = Sequential()
    model.add(Dense(units = 5, input_dim=775, activation="relu", \
                    kernel_initializer="uniform", kernel_regularizer=regularizers.l2(0.1)))
    model.add(Dropout(0.2))
    model.add(Dense(units = 5, init='uniform', activation='sigmoid'))
    model.compile( loss = "categorical_crossentropy", 
#                optimizer = keras.optimizers.SGD(lr=0.01, clipvalue=0.5), 
                optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
               metrics=['accuracy']
             )
    
  #  batch_size cannot be 1, at least contain all the classes for cross-entropy
    model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=18, epochs=100, callbacks=[stop_here_please])

    print(model.predict_classes(X_train))
    print(y_train)


#     print(history.losses)

    

#     model = Sequential()
#     model.add(LSTM(3, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dropout(0.2))
#     model.add(LSTM(10, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dense(50, activation='sigmoid'))
#     model.add(Dense(5, activation='sigmoid'))
#     model.compile(loss="categorical_crossentropy",
#                   optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
#                   metrics=['accuracy'])
#     model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=36, epochs=2000, callbacks=[history])



    scores = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    
    
print(np.mean(cvscores))







In [ ]:
print(model.predict_classes(X_test))
print(y_test)



In [ ]:
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
print(scaler.fit(data))
StandardScaler(copy=True, with_mean=True, with_std=True)
print(scaler.mean_)
print(scaler.transform(data))


# Baseline PCA + Logistic regression model

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  


samples = []
for s in range(epochs_std.shape[0]):
    X = epochs_std[s,:,:]
    pca = PCA(n_components=2)
    pca.fit(X)
    pca_com = pca.components_.flatten()
    samples.append(pca_com)
samples = np.asarray(samples)
# print(samples.shape)
# logistic = linear_model.LogisticRegression()
# logistic.fit(samples, np.asarray(labels))
# y_pred = logistic.predict(samples)


scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(samples):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = samples[train_idx]
    X_test = samples[test_idx]


    
    logistic = linear_model.LogisticRegression(C = 1e-5)
    logistic.fit(X_train, y_train)
    

    
    y_predict = logistic.predict(X_test)
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# Baseline PCA + NN model

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  


samples = []
for s in range(epochs_std.shape[0]):
    X = epochs_std[s,:,:]
    pca = PCA(n_components=2)
    pca.fit(X)
    pca_com = pca.components_.flatten()
    samples.append(pca_com)
samples = np.asarray(samples)
print(samples.shape)



cvscores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(samples):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = samples[train_idx]
    X_test = samples[test_idx]


    
    model = Sequential()
    model.add(Dense(units = 5, input_dim=1500, activation="relu", \
                    kernel_initializer="uniform", kernel_regularizer=regularizers.l2(0.1)))
    model.add(Dropout(0.2))
    model.add(Dense(units = 5, init='uniform', activation='sigmoid'))
    model.compile( loss = "categorical_crossentropy", 
#                optimizer = keras.optimizers.SGD(lr=0.01, clipvalue=0.5), 
                optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
               metrics=['accuracy']
             )
    
  #  batch_size cannot be 1, at least contain all the classes for cross-entropy
    model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=18, epochs=500, callbacks=[stop_here_please])


    
    scores = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    
    
print(np.mean(cvscores))

# Classic Common spatial patten + LDA

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP



# scores = []

# cv = ShuffleSplit(10, test_size=0.2, random_state=42)
# cv_split = cv.split(epochs_std)

# # Assemble a classifier
# lda = LinearDiscriminantAnalysis()
# csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# # Use scikit-learn Pipeline with cross_val_score function
# clf = Pipeline([('CSP', csp), ('LDA', lda)])
# scores = cross_val_score(clf, epochs_std, np.asarray(labels), cv=cv, n_jobs=1)

# # Printing the results
# class_balance = np.mean(labels == labels[0])
# class_balance = max(class_balance, 1. - class_balance)
# print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
#                                                           class_balance))


scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(epochs_std):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_std[train_idx]
    X_test = epochs_std[test_idx]
    print(X_train.shape)
    
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
    new_epochs = csp.fit_transform(X_train,  y_train)
    lda = LinearDiscriminantAnalysis()
    lda.fit(new_epochs, y_train)
    

    y_pred_csp = csp.transform(X_test)
    y_pred = lda.predict(y_pred_csp)
    
    print(y_pred)
    print(y_test)
    print(sklearn.metrics.accuracy_score(y_pred, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_pred, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(144, 61, 750)
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
['1' '4' '4' '3' '1' '3' '2' '1' '4' '3' '3' '1' '3' '2' '4' '1' '3' '4'
 '1' '4' '4' '3' '2' '2' '3' '3' '4' '3' '2' '2' '1' '3' '1' '4' '3' '3']
['1' '4' '3' '3' '1' '4' '1' '1' '4' '3' '3' '2' '4' '2' '3' '1' '4' '4'
 '3' '4' '3' '4' '2' '2' '4' '4' '4' '3' '1' '2' '1' '3' '1' '4' '3' '3']
0.6388888888888888
(144, 61, 750)
Estimating covariance using EMPIRICAL
Done.


C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
['2' '4' '2' '2' '2' '2' '2' '1' '2' '3' '2' '3' '4' '4' '4' '4' '3' '1'
 '3' '4' '2' '3' '3' '4' '1' '1' '1' '3' '1' '4' '1' '1' '1' '2' '2' '3']
['2' '3' '2' '2' '2' '2' '1' '1' '2' '3' '1' '3' '4' '3' '4' '4' '3' '1'
 '4' '4' '2' '3' '3' '4' '1' '1' '1' '4' '1' '4' '1' '1' '1' '2' '2' '3']
0.8333333333333334
(144, 61, 750)
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using EMPIRICAL
Done.
['1' '1' '1' '3' '4' '2' '2' '3' '3' '3' '1' '1' '3' '3' '1' '4' '3' '1'
 '3' '4' '2' '1' '2' '3' '1' '3' '1' '1' '2' '1' '1' '1' '3' '2' '1' '1']
['1' '1' '1' '3' '4' '2' '2' '3' '4' '3' '1' '2' '3' '3' '1' '4' '1' '3'
 '3' '4' '2' '4' '1' '3' '1' '4' '1' '1' '2' '1' '1' '2' '4' '2' '1' '1']
0.75
(144, 61, 750)
Estimating covariance using EMPIRICAL
Done.
Estima

In [49]:
a = ['2' '1' '3' '1' '1' '2' '4' '2' '3' '2' '3' '3' '2' '2' '3' '4' '2' '2'
 '3' '1' '3' '3' '1' '3' '2' '4' '2' '3' '1' '4' '4' '3' '3' '2' '4' '3']


1